In [ ]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

# CSV endpoint - downloads data as CSV instead of JSON
API_ENDPOINT = "https://data.cityofnewyork.us/resource/h9gi-nx95.csv"

def fetch_collision_data_csv(limit=1000):
    """
    Fetch collision data as CSV from NYC Open Data API
    CSV format is faster and easier to work with than JSON
    """

    # Calculate date range (last 7 days)
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

    # Build query parameters
    params = {
        '$limit': limit,
        '$order': 'crash_date DESC',
        '$where': f"crash_date between '{start_date}' and '{end_date}'"
    }

    print(f"🔍 Fetching collision data from NYC Open Data API (CSV format)...")
    print(f"   Date range: {start_date} to {end_date}")
    print(f"   Limit: {limit} records")

    try:
        # Make request
        response = requests.get(API_ENDPOINT, params=params, timeout=60)

        if response.status_code == 200:
            # Save raw CSV
            os.makedirs('data', exist_ok=True)
            filename = f"data/collisions_{start_date}_to_{end_date}.csv"

            with open(filename, 'wb') as f:
                f.write(response.content)

            # Read CSV with pandas
            df = pd.read_csv(filename)

            print(f"✅ Successfully fetched {len(df)} records")
            print(f"💾 Saved to {filename}")

            # Print summary
            print(f"\n📊 Summary:")
            print(f"   Total records: {len(df)}")
            print(f"   Columns: {len(df.columns)}")
            print(f"   Date range: {df['crash_date'].min()} to {df['crash_date'].max()}")

            if 'borough' in df.columns:
                print(f"\n   Collisions by Borough:")
                print(df['borough'].value_counts().to_string())

            return df

        else:
            print(f"❌ Error: {response.status_code}")
            print(f"   Message: {response.text}")
            return None

    except Exception as e:
        print(f"❌ Exception occurred: {str(e)}")
        return None

def main():
    """Main function"""
    print("=" * 60)
    print("NYC Motor Vehicle Collision Data Fetcher (CSV)")
    print("=" * 60)

    df = fetch_collision_data_csv(limit=1000)

    if df is not None:
        print("\n✅ ETL Complete!")
        print(f"   Check the 'data/' folder for CSV file")
        print(f"\n📋 Column names:")
        for col in df.columns:
            print(f"      - {col}")
    else:
        print("\n❌ ETL Failed - check error messages above")

if __name__ == "__main__":
    main()

NYC Motor Vehicle Collision Data Fetcher (CSV)
🔍 Fetching collision data from NYC Open Data API (CSV format)...
   Date range: 2026-01-14 to 2026-01-21
   Limit: 1000 records
✅ Successfully fetched 728 records
💾 Saved to data/collisions_2026-01-14_to_2026-01-21.csv

📊 Summary:
   Total records: 728
   Columns: 29
   Date range: 2026-01-14T00:00:00.000 to 2026-01-17T00:00:00.000

   Collisions by Borough:
borough
BROOKLYN         198
QUEENS           154
MANHATTAN        113
BRONX             96
STATEN ISLAND     30

✅ ETL Complete!
   Check the 'data/' folder for CSV file

📋 Column names:
      - crash_date
      - crash_time
      - borough
      - zip_code
      - latitude
      - longitude
      - location
      - on_street_name
      - off_street_name
      - cross_street_name
      - number_of_persons_injured
      - number_of_persons_killed
      - number_of_pedestrians_injured
      - number_of_pedestrians_killed
      - number_of_cyclist_injured
      - number_of_cyclist_killed

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

API = "https://data.cityofnewyork.us/resource/h9gi-nx95.csv"

# Get last 7 days
end = datetime.now().strftime('%Y-%m-%d')
start = (datetime.now() - timedelta(7)).strftime('%Y-%m-%d')

# Fetch data
params = {'$limit': 1000, '$where': f"crash_date between '{start}' and '{end}'"}
response = requests.get(API, params=params)

# Save
filename = f"data/collisions_{start}_to_{end}.csv"
open(filename, 'wb').write(response.content)

# Display
df = pd.read_csv(filename)
print(f"✅ {len(df)} records saved to {filename}")
print(df['borough'].value_counts())

✅ 728 records saved to data/collisions_2026-01-14_to_2026-01-21.csv
borough
BROOKLYN         198
QUEENS           154
MANHATTAN        113
BRONX             96
STATEN ISLAND     30
Name: count, dtype: int64
